In [14]:
import pandas as pd

In [15]:
df_raw = pd.read_csv("./player-data-full.csv")
df_raw.head()

/tmp/ipykernel_3253666/1018638222.py:1: DtypeWarning: Columns (73) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("./player-data-full.csv")


,player_id,version,name,full_name,description,image,height_cm,weight_kg,dob,positions,...,composure,defensive_awareness,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,play_styles
0,239085,2024-07-08,Erling Haaland,Erling Braut Haaland,"Erling Haaland (Erling Braut Haaland, born 21 ...",https://cdn.sofifa.net/players/239/085/24_120.png,195,94,2000-07-21,ST,...,87,38,47,29,7,14,13,11,7.0,"Acrobatic +,Power Header,Quick Step"
1,231747,2024-07-08,Kylian Mbappé,Kylian Mbappé Lottin,"Kylian Mbappé (Kylian Mbappé Lottin, born 20 D...",https://cdn.sofifa.net/players/231/747/24_120.png,182,75,1998-12-20,"ST,LW",...,88,26,34,32,13,5,7,11,6.0,"Quick Step +,Finesse Shot,Rapid,Flair,Trivela,..."
2,192985,2024-07-08,Kevin De Bruyne,Kevin De Bruyne,Kevin De Bruyne (born 28 June 1991) is a Belgi...,https://cdn.sofifa.net/players/192/985/24_120.png,181,75,1991-06-28,"CM,CAM",...,88,66,70,53,15,13,5,10,13.0,"Incisive Pass +,Dead Ball,Pinged Pass,Long Bal..."
3,231866,2024-07-08,Rodri,Rodrigo Hernández Cascante,Rodri (born 22 June 1996) is a Spanish footbal...,https://cdn.sofifa.net/players/231/866/24_120.png,191,82,1996-06-22,"CDM,CM",...,90,92,87,78,10,10,7,14,8.0,"Power Shot +,Long Ball Pass,Bruiser,Aerial,Lon..."
4,202126,2024-07-08,Harry Kane,Harry Kane,Harry Kane (born 28 July 1993) is a British fo...,https://cdn.sofifa.net/players/202/126/24_120.png,188,85,1993-07-28,ST,...,92,46,46,38,8,10,11,14,11.0,"Incisive Pass +,Long Ball Pass,First Touch,Tri..."


In [16]:
def inspeccionar(df: pd.DataFrame) -> pd.DataFrame:
    # print("Dimensiones del DataFrame:", df.shape)
    # print("Tipos de datos:\n", df.dtypes)
    # print("Columnas con valores nulos:\n", df.columns[df.isnull().any()])
    return df

In [17]:
def calcular_edad(df: pd.DataFrame) -> pd.DataFrame:
    df['dob'] = pd.to_datetime(df['dob'])
    df['age'] = (pd.to_datetime('2025-07-16') - df['dob']).dt.days // 365
    return df


In [18]:
print(df_raw.columns)

Index(['player_id', 'version', 'name', 'full_name', 'description', 'image',
       'height_cm', 'weight_kg', 'dob', 'positions', 'overall_rating',
       'potential', 'value', 'wage', 'preferred_foot', 'weak_foot',
       'skill_moves', 'international_reputation', 'work_rate', 'body_type',
       'real_face', 'release_clause', 'specialities', 'club_id', 'club_name',
       'club_league_id', 'club_league_name', 'club_logo', 'club_rating',
       'club_position', 'club_kit_number', 'club_joined',
       'club_contract_valid_until', 'country_id', 'country_name',
       'country_league_id', 'country_league_name', 'country_flag',
       'country_rating', 'country_position', 'country_kit_number', 'crossing',
       'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'fk_accuracy', 'long_passing', 'ball_control',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
 

In [19]:

def eliminar_columnas_innecesarias(df: pd.DataFrame) -> pd.DataFrame:
    to_drop = ['version', 'description', 'image',\
       'club_id', 'club_league_id','club_logo',\
       'club_kit_number','country_id','country_league_id', 'country_flag']
    
    return df.drop(columns=to_drop, errors='ignore')


In [ ]:
def tratar_nulos(df: pd.DataFrame) -> pd.DataFrame:
    # Make a copy to avoid SettingWithCopyWarning
    df = df.copy()
    
    columnas_clave = ['club_name', 'club_league_name', 'positions']
    df = df.dropna(subset=columnas_clave)
    
    
    df['release_clause'] = df['release_clause'].fillna(0)
    
    # TODO: Validate threshold
    threshold = len(df) * 0.5
    df = df.dropna(axis=1, thresh=threshold)
    
    return df


In [21]:
def parse_monetary_string(monetary_str: str) -> float:
    if pd.isna(monetary_str) or monetary_str == '':
        return 0.0
    
    # Remove € symbol
    value_str = str(monetary_str).replace('€', '')
    
    # Check if it ends with K or M
    if value_str.endswith('K'):
        # Remove K and multiply by 1000
        number = float(value_str[:-1])
        return number * 1000
    elif value_str.endswith('M'):
        # Remove M and multiply by 1000000
        number = float(value_str[:-1])
        return number * 1000000
    else:
        # If no K or M, assume it's already in the base unit
        return float(value_str)

def convertir_salarios_y_valores(df: pd.DataFrame) -> pd.DataFrame:
    # Parse monetary columns
    df['value_eur'] = df['value'].apply(parse_monetary_string)
    df['wage_eur'] = df['wage'].apply(parse_monetary_string)
    df['release_clause_eur'] = df['release_clause'].apply(parse_monetary_string)
    
    return df


In [22]:
def convertir_tipos(df: pd.DataFrame) -> pd.DataFrame:
    
    df['value_eur'] = df['value_eur'].astype(float)
    df['wage_eur'] = df['wage_eur'].astype(float)
    df['release_clause_eur'] = df['release_clause_eur'].astype(float)
    return df


In [23]:
def agregar_columnas_utiles(df: pd.DataFrame) -> pd.DataFrame:
    df['grupo_edad'] = pd.cut(
        df['age'], 
        bins=[0, 21, 27, 33, 51], 
        labels=['joven', 'maduro', 'experimentado', 'veterano'], 
        right=False
    )
    
    df['es_promesa'] = ((df['age'] <= 21) & (df['potential'] >= 80)).astype(int)
    return df


In [24]:
def procesar_posiciones(df: pd.DataFrame) -> pd.DataFrame:
    
    df['posicion_principal'] = df['positions'].str.split(',').str[0]
    
    posicion_mapping = {
        'GK': 'Arquero',
        'CB': 'Defensor central',
        'RB': 'Lateral',
        'LB': 'Lateral',
        'RWB': 'Lateral',
        'LWB': 'Lateral',
        'CDM': 'Volante defensivo',
        'CM': 'Volante',
        'CAM': 'Volante',
        'RM': 'Volante',
        'LM': 'Volante',
        'ST': 'Delantero',
        'CF': 'Delantero',
        'RW': 'Extremo',
        'LW': 'Extremo'
    }
    
    df['posicion_simplificada'] = df['posicion_principal'].map(posicion_mapping)
    
    return df


In [25]:
def verificar_final(df: pd.DataFrame) -> pd.DataFrame:
    # print("Forma final del DataFrame:", df.shape)
    # print("\nColumnas con valores nulos:")
    nulos = df.isnull().sum()
    columnas_con_nulos = nulos[nulos > 0]
    
    # if len(columnas_con_nulos) == 0:
    #     print("No hay columnas con valores nulos")
    # else:
    #     for columna, cantidad in columnas_con_nulos.items():
    #         print(f"  {columna}: {cantidad} valores nulos")
    return df

In [26]:
df = (
    df_raw
    .pipe(inspeccionar)
    .pipe(calcular_edad)
    .pipe(eliminar_columnas_innecesarias)
    .pipe(tratar_nulos)
    .pipe(convertir_salarios_y_valores)
    .pipe(convertir_tipos)
    .pipe(agregar_columnas_utiles)
    .pipe(procesar_posiciones)
    .pipe(verificar_final)
)

df.head()

,player_id,name,full_name,height_cm,weight_kg,dob,positions,overall_rating,potential,value,...,gk_reflexes,play_styles,age,value_eur,wage_eur,release_clause_eur,grupo_edad,es_promesa,posicion_principal,posicion_simplificada
0,239085,Erling Haaland,Erling Braut Haaland,195,94,2000-07-21,ST,91,94,€185M,...,7.0,"Acrobatic +,Power Header,Quick Step",25,185000000.0,340000.0,356100000.0,maduro,0,ST,Delantero
1,231747,Kylian Mbappé,Kylian Mbappé Lottin,182,75,1998-12-20,"ST,LW",91,94,€181.5M,...,6.0,"Quick Step +,Finesse Shot,Rapid,Flair,Trivela,...",26,181500000.0,230000.0,349400000.0,maduro,0,ST,Delantero
2,192985,Kevin De Bruyne,Kevin De Bruyne,181,75,1991-06-28,"CM,CAM",91,91,€103M,...,13.0,"Incisive Pass +,Dead Ball,Pinged Pass,Long Bal...",34,103000000.0,350000.0,190600000.0,veterano,0,CM,Volante
3,231866,Rodri,Rodrigo Hernández Cascante,191,82,1996-06-22,"CDM,CM",90,92,€129.5M,...,8.0,"Power Shot +,Long Ball Pass,Bruiser,Aerial,Lon...",29,129500000.0,260000.0,249300000.0,experimentado,0,CDM,Volante defensivo
4,202126,Harry Kane,Harry Kane,188,85,1993-07-28,ST,90,90,€119.5M,...,11.0,"Incisive Pass +,Long Ball Pass,First Touch,Tri...",31,119500000.0,170000.0,197200000.0,experimentado,0,ST,Delantero
